# SQL-проект

Цель исследования изучить данные только приобретенного сервиса для чтения книг, для этого будет произведено исследование , кетоторых важных метрик.

In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

'books', 'authors', 'publishers', 'ratings', 'reviews'

## upload

### upload books

In [6]:
query = '''
            SELECT * FROM books
        '''

In [7]:
books = pd.io.sql.read_sql(query, con = engine)

In [8]:
display(books.head(1))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


### upload authors

In [9]:
query = '''
            SELECT * FROM authors
        '''

In [10]:
authors = pd.io.sql.read_sql(query, con = engine)

In [11]:
display(authors.head(1))

,author_id,author
0,1,A.S. Byatt


### upload publishers

In [12]:
query = '''
            SELECT * FROM publishers
        '''

In [13]:
publishers = pd.io.sql.read_sql(query, con = engine)

In [14]:
display(publishers.head(1))

,publisher_id,publisher
0,1,Ace


### upload ratings

In [15]:
query = '''
            SELECT * FROM ratings
        '''

In [16]:
ratings = pd.io.sql.read_sql(query, con = engine)

In [17]:
display(ratings.head(1))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


### upload reviews

In [18]:
query = '''
            SELECT * FROM reviews
        '''

In [19]:
reviews = pd.io.sql.read_sql(query, con = engine)

In [20]:
display(reviews.head(1))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## Task one
Посчитайте, сколько книг вышло после 1 января 2000 года;

In [21]:
task_one = '''
            SELECT COUNT(*)
            FROM books
            WHERE publication_date::date >='2000-01-01'
        ''' 
one = pd.io.sql.read_sql(task_one, con = engine)
display(one)

,count
0,821


## Task two
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [22]:
two = '''
            SELECT p.title,COUNT(DISTINCT review_id),
                    AVG(rating) AS avg_rating
                    
            FROM books as p 
            JOIN ratings as r ON p.book_id = r.rating_id
            JOIN reviews as rev ON p.book_id = rev.book_id
            GROUP BY rating ,p.title 
        ''' 
two = pd.io.sql.read_sql(two, con = engine)
display(two)




,title,count,avg_rating
0,Jane Eyre,5,1.0
1,Midnight's Children,4,1.0
2,My Side of the Mountain (Mountain #1),2,1.0
3,Suzanne's Diary for Nicholas,3,1.0
4,Tropic of Cancer,2,1.0
...,...,...,...
988,Wizard's First Rule (Sword of Truth #1),4,5.0
989,Women,2,5.0
990,Xenocide (Ender's Saga #3),3,5.0
991,Year of Wonders,4,5.0


Для каждой книги посчитайте количество обзоров и среднюю оценку

In [23]:
two_var_one = '''

SELECT p.book_id,p.title,COUNT(review_id),
                    AVG(rating) AS avg_rating
                    
            FROM books as p 
            left outer JOIN ratings as r ON p.book_id = r.rating_id
            left outer JOIN reviews as rev ON p.book_id = rev.book_id
            GROUP BY p.book_id,rating ,p.title 
             
        ''' 
two_var_one = pd.io.sql.read_sql(two_var_one, con = engine)
display(two_var_one)

,book_id,title,count,avg_rating
0,786,The Lost Continent: Travels in Small Town America,2,3.0
1,163,Corelli's Mandolin,3,3.0
2,583,Something Blue (Darcy & Rachel #2),4,4.0
3,590,Special Topics in Calamity Physics,2,4.0
4,460,My Side of the Mountain (Mountain #1),2,1.0
...,...,...,...,...
995,444,Moneyball: The Art of Winning an Unfair Game,3,5.0
996,563,Shopaholic & Baby (Shopaholic #5),3,3.0
997,791,The Magus,2,3.0
998,398,Little Town on the Prairie (Little House #7),2,3.0


## Task three
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [24]:
three = ''' select p.publisher,count(book_id) as count_book
from books as b
join publishers as p on  b.publisher_id = p.publisher_id 
where num_pages >50
group by p.publisher
order by count_book desc 
limit 1
 ''' 
three = pd.io.sql.read_sql(three, con = engine)
display(three)

,publisher,count_book
0,Penguin Books,42


## Task four
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [25]:
four = ''' 
select a.author as author_name,
    avg(r.rating) as rating 
from authors as a
--авторы + книги + рейтинг
join books as b on a.author_id = b.author_id 
join ratings as r on b.book_id = r.book_id
group by author_name
having count(rating)>=50
order by rating desc 
limit 1
'''
four = pd.io.sql.read_sql(four, con = engine)
display(four)

,author_name,rating
0,Diana Gabaldon,4.3


In [26]:
four_var_one = ''' 
select b.book_id,a.author, avg(rating) as avg_rating
from authors a 
join books as b on a.author_id  = b.author_id 
join ratings as r on b.book_id = r.book_id 
where b.book_id  in 
(
select b.book_id
from books as b 
join ratings as r on b.book_id = r.book_id 
group by b.book_id 
having count(rating) >= 50)
group by b.book_id,a.author
order by avg_rating desc 
'''
four_var_one = pd.io.sql.read_sql(four_var_one, con = engine)
display(four_var_one)

,book_id,author,avg_rating
0,302,J.K. Rowling/Mary GrandPré,4.414634
1,722,J.R.R. Tolkien,4.391892
2,299,J.K. Rowling/Mary GrandPré,4.287500
3,656,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,300,J.K. Rowling/Mary GrandPré,4.246575
5,399,Louisa May Alcott,4.192308
6,301,J.K. Rowling/Mary GrandPré,4.186667
7,750,J.R.R. Tolkien,4.125000
8,779,Rick Riordan,4.080645
9,405,William Golding,3.901408


J.K. Rowling/Mary GrandPré у не самый высокий рейтинг

In [27]:
five = ''' 
select avg(c_id)
from(
select count(review_id) as c_id
from reviews
where username  in(
select username
from ratings
group by username
having count (rating_id) > 50)
group by username) as one
'''
five = pd.io.sql.read_sql(five, con = engine)
display(five)

,avg
0,24.333333


## Выводы

821 книга вышла после 1 января 2000 года;

Создана таблица для каждой книги с количеством обзоров и средней оценкой;

Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books

Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré;

Среднее количество обзоров от пользователей 24